In [1]:
# One time installation of packages
#!pip install pyreadstat
#!pip install pprint
#!pip install re
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install matplotlib_venn
#!pip install sqlite3

## Import the Python Modules needed for this work

In [2]:
import pandas as pd
import sqlite3
import re
import sys
sys.path.append(r'/Users/stepwate/Python Codes/Reusable Code')
import trello_generic as tg

# Set number of rows and columns to visualise 
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)

## Import this month's CSV file for the Brand Tracker

In [3]:
brand_tracker1=pd.read_csv(r"/Users/stepwate/Downloads/BritBox-Brand-Tracker-December.csv")
This_month='Dec19'
brand_battery=['ITV Hub Plus',\
'ITV Hub',\
'BBC iPlayer',\
'All4',\
'My5',\
'YouTube',\
'Netflix',\
'Amazon Prime',\
'BritBox',\
'Hayu',\
'Now TV',\
'Apple TV Plus',\
'Apple TV',\
'Disney Plus',\
'Disney Life/Disney Masterbrand',\
'None',\
]

## Load dataframe into a (local) SQL database

In [4]:
conn = sqlite3.connect('SQL_connection1.db') #Create a connection object

# Put the pandas table into the connection object. Drop the table that already exists
try:
    query=conn.cursor()
    query.execute('''drop table Brand_Tracker_{}_1'''.format(This_month))
    query.fetchall()
    query.close()
except:
    pass

brand_tracker1.to_sql('Brand_Tracker_{}_1'.format(This_month),con=conn)



## Run SQL queries to clean the data as appropriate
#### Some useful syntax:
##### To run SQL statements 
query=conn.cursor()
query.execute('''<some SQL>''')
query.fetchall()
query.close()

##### To query directly via pandas
pd.read_sql_query('''<some SQL>''', conn)

##### Or read results directly into pandas dataframe
Mynewtable = pd.read_sql_query('''<some SQL>''', conn)

### Script 1: renames variables 

In [5]:
dropped_script='''
,Q5a1_1 as "ITP free v1- BBC iPlayer"
,Q5a1_2 as "ITP free v1- ITV Hub"
,Q5a1_3 as "ITP free v1- All 4"
,Q5a1_4 as "ITP free v1- My5"
,Q5a1_6 as "ITP free v1- YouTube"
,Q5a1_7 as "ITP free v1- None of these"
,Q5a2_1 as "ITP free v2 scale- BBC iPlayer"
,Q5a2_2 as "ITP free v2 scale- ITV Hub"
,Q5a2_3 as "ITP free v2 scale- All 4"
,Q5a2_4 as "ITP free v2 scale- My5"
,Q5a2_6 as "ITP free v2 scale- YouTube"
,Q5b1_1 as "ITP paid v1- None of these"
,Q5b1_2 as "ITP paid v1- Netflix"
,Q5b1_3 as "ITP paid v1- Amazon Prime Video"
,Q5b1_4 as "ITP paid v1- BritBox"
,Q5b1_5 as "ITP paid v1- Hayu"
,Q5b1_6 as "ITP paid v1- Now TV"
,Q5b1_7 as "ITP paid v1- Disney Plus"
,Q5b1_8 as "ITP paid v1- Apple TV +"
,Q5b1_9 as "ITP paid v1- None of these"
'''

In [6]:
sql_query_string='''
create table Brand_Tracker_{}_2 as select 
'{}' as Month,
'{}_'||cast(row_number() over () as string) as Response_ID,
weight,
profile_gender as "Gender"
,profile_itv_coded as "Region"
,age3 as "Age"
,profile_socialgrade_ciew as "Social Grade"
,children_aged_16_under as "Children in HH"
,Q1_1_1 as "Unaided awareness_Option 1"
,Q1_2_1 as "Unaided awareness- Option 2"
,Q1_3_1 as "Unaided awareness- Option 3"
,Q1_4_1 as "Unaided awareness- Option 4"
,Q1_5_1 as "Unaided awareness- Option 5"
,Q1_first as "Unaided awareness- Q1_first recoded variable"
,Q1_recoded_1 as "Unaided awareness- ITV Hub+"
,Q1_recoded_2 as "Unaided awareness- ITV player/hub/OD"
,Q1_recoded_3 as "Unaided awareness- Netflix"
,Q1_recoded_4 as "Unaided awareness- Amazon Prime Video"
,Q1_recoded_5 as "Unaided awareness- BritBox"
,Q1_recoded_6 as "Unaided awareness- Hayu"
,Q1_recoded_7 as "Unaided awareness- Now TV"
,Q1_recoded_8 as "Unaided awareness- Apple TV+"
,Q1_recoded_9 as "Unaided awareness- Apple TV/Apple Masterbrand"
,Q1_recoded_10 as "Unaided awareness- Disney Plus"
,Q1_recoded_11 as "Unaided awareness- Disney Life/Disney Masterbrand"
,Q1_recoded_95 as "Unaided awareness- Other"
,Q2_1 as "Experience with brand- BBC iPlayer"
,Q2_2 as "Experience with brand- ITV Hub"
,Q2_3 as "Experience with brand- ITV Hub+ "
,Q2_4 as "Experience with brand- All 4"
,Q2_5 as "Experience with brand- My5"
,Q2_6 as "Experience with brand- Hayu"
,Q2_7 as "Experience with brand- Netflix"
,Q2_8 as "Experience with brand- Amazon Prime Video"
,Q2_9 as "Experience with brand- YouTube"
,Q2_10 as "Experience with brand- Now TV"
,Q2_11 as "Experience with brand- BritBox"
,Q2_12 as "Experience with brand- Disney Plus"
,Q2_13 as "Experience with brand- Apple TV+"
,Q3_1 as "Service usage- BBC iPlayer"
,Q3_2 as "Service usage- ITV Hub"
,Q3_3 as "Service usage- ITV Hub+"
,Q3_4 as "Service usage- All 4"
,Q3_5 as "Service usage- My5"
,Q3_6 as "Service usage- Hayu"
,Q3_7 as "Service usage- Netflix"
,Q3_8 as "Service usage- Amazon Prime Video"
,Q3_9 as "Service usage- YouTube"
,Q3_10 as "Service usage- Now TV (subscription service)"
,Q3_11 as "Service usage- BritBox"
,Q3_12 as "Service usage- Disney Plus"
,Q3_13 as "Service usage- Apple TV +"
,Q4_1 as "Current SVOD subscriptions- ITV Hub+"
,Q4_2 as "Current SVOD subscriptions- Netflix"
,Q4_3 as "Current SVOD subscriptions- Amazon Prime Video"
,Q4_4 as "Current SVOD subscriptions- BritBox"
,Q4_5 as "Current SVOD subscriptions- Hayu"
,Q4_6 as "Current SVOD subscriptions- Now TV"
,Q4_7 as "Current SVOD subscriptions- Disney Plus"
,Q4_8 as "Current SVOD subscriptions- Apple TV+"
,Q4_9 as "Current SVOD subscriptions- Other (please specify)"
,Q4_10 as "Current SVOD subscriptions- None"
,Q4_other as "Current SVOD subscriptions- Other1"
,Q5_1 as "How pay for SVOD subscriptions- ITV Hub+"
,Q5_2 as "How pay for SVOD subscriptions- Netflix"
,Q5_3 as "How pay for SVOD subscriptions- Amazon Prime Video"
,Q5_4 as "How pay for SVOD subscriptions- BritBox"
,Q5_5 as "How pay for SVOD subscriptions- Hayu"
,Q5_6 as "How pay for SVOD subscriptions- Now TV"
,Q5_7 as "How pay for SVOD subscriptions- Disney Plus"
,Q5_8 as "How pay for SVOD subscriptions- Apple TV+"

,Q5b2_1 as "ITP paid v2 scale- ITV Hub+"
,Q5b2_2 as "ITP paid v2 scale- Netflix"
,Q5b2_3 as "ITP paid v2 scale- Amazon Prime"
,Q5b2_4 as "ITP paid v2 scale- BritBox"
,Q5b2_5 as "ITP paid v2 scale- Hayu"
,Q5b2_6 as "ITP paid v2 scale- Now TV"
,Q5b2_7 as "ITP paid v2 scale- Disney Plus"
,Q5b2_8 as "ITP paid v2 scale- Apple TV"
,Q6a_1_1 as "Barriers free services- BBC iPlayer - Lack of information"
,Q6a_1_2 as "Barriers free services- BBC iPlayer - No shows"
,Q6a_1_3 as "Barriers free services- BBC iPlayer - Get all I need"
,Q6a_1_4 as "Barriers free services- BBC iPlayer - Product features"
,Q6a_1_5 as "Barriers free services- BBC iPlayer - Content elsewhere"
,Q6a_1_6 as "Barriers free services- BBC iPlayer - British content "
,Q6a_1_7 as "Barriers free services- BBC iPlayer - Programme/film awareness"
,Q6a_1_8 as "Barriers free services- BBC iPlayer - No access"
,Q6a_1_9 as "Barriers free services- BBC iPlayer - Don't want VOD"
,Q6a_1_10 as "Barriers free services- BBC iPlayer - None of these"
,Q6a_2_1 as "Barriers free services- ITV Hub - Lack of information"
,Q6a_2_2 as "Barriers free services- ITV Hub - No shows"
,Q6a_2_3 as "Barriers free services- ITV Hub - Get all I need"
,Q6a_2_4 as "Barriers free services- ITV Hub - Product features"
,Q6a_2_5 as "Barriers free services- ITV Hub - Content elsewhere"
,Q6a_2_6 as "Barriers free services- ITV Hub - British content "
,Q6a_2_7 as "Barriers free services- ITV Hub - Programme/film awareness"
,Q6a_2_8 as "Barriers free services- ITV Hub - No access"
,Q6a_2_9 as "Barriers free services- ITV Hub - Don't want VOD"
,Q6a_2_10 as "Barriers free services- ITV Hub - None of these"
,Q6a_3_1 as "Barriers free services- All4 - Lack of information"
,Q6a_3_2 as "Barriers free services- All4  - No shows"
,Q6a_3_3 as "Barriers free services- All4  - Get all I need"
,Q6a_3_4 as "Barriers free services- All4  - Product features"
,Q6a_3_5 as "Barriers free services- All4  - Content elsewhere"
,Q6a_3_6 as "Barriers free services- All4  - British content "
,Q6a_3_7 as "Barriers free services- All4  - Programme/film awareness"
,Q6a_3_8 as "Barriers free services- All4  - No access"
,Q6a_3_9 as "Barriers free services- All4  - Don't want VOD"
,Q6a_3_10 as "Barriers free services- All4  - None of these"
,Q6a_4_1 as "Barriers free services- My5 - Lack of information"
,Q6a_4_2 as "Barriers free services- My5  - No shows"
,Q6a_4_3 as "Barriers free services- My5 - Get all I need"
,Q6a_4_4 as "Barriers free services- My5  - Product features"
,Q6a_4_5 as "Barriers free services- My5  - Content elsewhere"
,Q6a_4_6 as "Barriers free services- My5  - British content "
,Q6a_4_7 as "Barriers free services- My5  - Programme/film awareness"
,Q6a_4_8 as "Barriers free services- My5  - No access"
,Q6a_4_9 as "Barriers free services- My5  - Don't want VOD"
,Q6a_4_10 as "Barriers free services- My5  - None of these"
,Q6a_5_1 as "Barriers free services- YouTube - Lack of information"
,Q6a_5_2 as "Barriers free services- YouTube  - No shows"
,Q6a_5_3 as "Barriers free services- YouTube - Get all I need"
,Q6a_5_4 as "Barriers free services- YouTube  - Product features"
,Q6a_5_5 as "Barriers free services- YouTube  - Content elsewhere"
,Q6a_5_6 as "Barriers free services- YouTube  - British content "
,Q6a_5_7 as "Barriers free services- YouTube  - Programme/film awareness"
,Q6a_5_8 as "Barriers free services- YouTube  - No access"
,Q6a_5_9 as "Barriers free services- YouTube  - Don't want VOD"
,Q6a_5_10 as "Barriers free services- YouTube  - None of these"
,Q6b_1_1 as "Barriers paid services- ITV Hub+ - Lack of information"
,Q6b_1_2 as "Barriers paid services- ITV Hub+  - No shows"
,Q6b_1_3 as "Barriers paid services- ITV Hub+ - Get all I need"
,Q6b_1_4 as "Barriers paid services- ITV Hub+  - Product features"
,Q6b_1_5 as "Barriers paid services- ITV Hub+  - Content elsewhere"
,Q6b_1_6 as "Barriers paid services- ITV Hub+  - British content "
,Q6b_1_7 as "Barriers paid services- ITV Hub+  - Programme/film awareness"
,Q6b_1_8 as "Barriers paid services- ITV Hub+  - No access"
,Q6b_1_9 as "Barriers paid services- ITV Hub+  - Don't want SVOD"
,Q6b_1_10 as "Barriers paid services- ITV Hub+ - Expensive"
,Q6b_1_11 as "Barriers paid services- ITV Hub+  - None of these"
,Q6b_1_12 as "Barriers paid services- ITV Hub+ - Don’t want another SVOD"
,Q6b_2_1 as "Barriers paid services- Netflix - Lack of information"
,Q6b_2_2 as "Barriers paid services- Netflix - No shows"
,Q6b_2_3 as "Barriers paid services- Netflix - Get all I need"
,Q6b_2_4 as "Barriers paid services- Netflix - Product features"
,Q6b_2_5 as "Barriers paid services- Netflix  - Content elsewhere"
,Q6b_2_6 as "Barriers paid services- Netflix - British content "
,Q6b_2_7 as "Barriers paid services- Netflix - Programme/film awareness"
,Q6b_2_8 as "Barriers paid services- Netflix - No access"
,Q6b_2_9 as "Barriers paid services- Netflix - Don't want SVOD"
,Q6b_2_10 as "Barriers paid services- Netflix - Expensive"
,Q6b_2_11 as "Barriers paid services- Netflix  - None of these"
,Q6b_2_12 as "Barriers paid services- Netflix - Don’t want another SVOD"
,Q6b_3_1 as "Barriers paid services- Amazon Prime - Lack of information"
,Q6b_3_2 as "Barriers paid services- Amazon Prime - No shows"
,Q6b_3_3 as "Barriers paid services- Amazon Prime - Get all I need"
,Q6b_3_4 as "Barriers paid services- Amazon Prime - Product features"
,Q6b_3_5 as "Barriers paid services- Amazon Prime  - Content elsewhere"
,Q6b_3_6 as "Barriers paid services- Amazon Prime - British content "
,Q6b_3_7 as "Barriers paid services- Amazon Prime - Programme/film awareness"
,Q6b_3_8 as "Barriers paid services- Amazon Prime - No access"
,Q6b_3_9 as "Barriers paid services- Amazon Prime - Don't want SVOD"
,Q6b_3_10 as "Barriers paid services- Amazon Prime - Expensive"
,Q6b_3_11 as "Barriers paid services- Amazon Prime - None of these"
,Q6b_3_12 as "Barriers paid services- Amazon Prime - Don’t want another SVOD"
,Q6b_4_1 as "Barriers paid services- BritBox - Lack of information"
,Q6b_4_2 as "Barriers paid services- BritBox - No shows"
,Q6b_4_3 as "Barriers paid services- BritBox - Get all I need"
,Q6b_4_4 as "Barriers paid services- BritBox - Product features"
,Q6b_4_5 as "Barriers paid services- BritBox - Content elsewhere"
,Q6b_4_6 as "Barriers paid services- BritBox - British content "
,Q6b_4_7 as "Barriers paid services- BritBox - Programme/film awareness"
,Q6b_4_8 as "Barriers paid services- BritBox - No access"
,Q6b_4_9 as "Barriers paid services- BritBox - Don't want SVOD"
,Q6b_4_10 as "Barriers paid services- BritBox - Expensive"
,Q6b_4_11 as "Barriers paid services- BritBox - None of these"
,Q6b_4_12 as "Barriers paid services- BritBox - Don’t want another SVOD"
,Q6b_5_1 as "Barriers paid services- Hayu - Lack of information"
,Q6b_5_2 as "Barriers paid services- Hayu - No shows"
,Q6b_5_3 as "Barriers paid services- Hayu - Get all I need"
,Q6b_5_4 as "Barriers paid services- Hayu - Product features"
,Q6b_5_5 as "Barriers paid services- Hayu - Content elsewhere"
,Q6b_5_6 as "Barriers paid services- Hayu - British content "
,Q6b_5_7 as "Barriers paid services- Hayu - Programme/film awareness"
,Q6b_5_8 as "Barriers paid services- Hayu - No access"
,Q6b_5_9 as "Barriers paid services- Hayu - Don't want SVOD"
,Q6b_5_10 as "Barriers paid services- Hayu - Expensive"
,Q6b_5_11 as "Barriers paid services- Hayu - None of these"
,Q6b_5_12 as "Barriers paid services- Hayu - Don’t want another SVOD"
,Q6b_6_1 as "Barriers paid services- Now TV - Lack of information"
,Q6b_6_2 as "Barriers paid services- Now TV - No shows"
,Q6b_6_3 as "Barriers paid services- Now TV - Get all I need"
,Q6b_6_4 as "Barriers paid services- Now TV - Product features"
,Q6b_6_5 as "Barriers paid services- Now TV - Content elsewhere"
,Q6b_6_6 as "Barriers paid services- Now TV - British content "
,Q6b_6_7 as "Barriers paid services- Now TV - Programme/film awareness"
,Q6b_6_8 as "Barriers paid services- Now TV - No access"
,Q6b_6_9 as "Barriers paid services- Now TV - Don't want SVOD"
,Q6b_6_10 as "Barriers paid services- Now TV - Expensive"
,Q6b_6_11 as "Barriers paid services- Now TV - None of these"
,Q6b_6_12 as "Barriers paid services- Now TV - Don’t want another SVOD"
,Q6b_7_1 as "Barriers paid services- Disney Plus - Lack of information"
,Q6b_7_2 as "Barriers paid services- Disney Plus - No shows"
,Q6b_7_3 as "Barriers paid services- Disney Plus - Get all I need"
,Q6b_7_4 as "Barriers paid services- Disney Plus - Product features"
,Q6b_7_5 as "Barriers paid services- Disney Plus - Content elsewhere"
,Q6b_7_6 as "Barriers paid services- Disney Plus - British content "
,Q6b_7_7 as "Barriers paid services- Disney Plus - Programme/film awareness"
,Q6b_7_8 as "Barriers paid services- Disney Plus - No access"
,Q6b_7_9 as "Barriers paid services- Disney Plus - Don't want SVOD"
,Q6b_7_10 as "Barriers paid services- Disney Plus - Expensive"
,Q6b_7_11 as "Barriers paid services- Disney Plus - None of these"
,Q6b_7_12 as "Barriers paid services- Disney Plus - Don’t want another SVOD"
,Q6b_8_1 as "Barriers paid services- Apple TV - Lack of information"
,Q6b_8_2 as "Barriers paid services- Apple TV - No shows"
,Q6b_8_3 as "Barriers paid services- Apple TV - Get all I need"
,Q6b_8_4 as "Barriers paid services- Apple TV - Product features"
,Q6b_8_5 as "Barriers paid services- Apple TV - Content elsewhere"
,Q6b_8_6 as "Barriers paid services- Apple TV - British content "
,Q6b_8_7 as "Barriers paid services- Apple TV - Programme/film awareness"
,Q6b_8_8 as "Barriers paid services- Apple TV - No access"
,Q6b_8_9 as "Barriers paid services- Apple TV - Don't want SVOD"
,Q6b_8_10 as "Barriers paid services- Apple TV - Expensive"
,Q6b_8_11 as "Barriers paid services- Apple TV - None of these"
,Q6b_8_12 as "Barriers paid services- Apple TV - Don’t want another SVOD"
,Q8_1 as "Likeability - BritBox"
,Q8_2 as "Likeability - Netflix"
,Q8_3 as "Likeability - Amazon Prime "
,Q8_4 as "Likeability - Now TV"
,Q8_5 as "Likeability - Disney Plus"
,Q8_6 as "Likeability - Apple TV"
,Q9_1 as "Recommendation- BritBox"
,Q9_2 as "Recommendation- Netflix"
,Q9_3 as "Recommendation- Amazon Prime "
,Q9_4 as "Recommendation- Now TV"
,Q9_5 as "Recommendation- Disney Plus"
,Q9_6 as "Recommendation- Apple TV"
,Q10_1 as "Uniqueness- BritBox"
,Q10_2 as "Uniqueness- Netflix"
,Q10_3 as "Uniqueness- Amazon Prime "
,Q10_4 as "Uniqueness- Now TV"
,Q10_5 as "Uniqueness- Disney Plus"
,Q10_6 as "Uniqueness- Apple TV"
,Q11_1 as "Meets needs- BritBox"
,Q11_2 as "Meets needs- Netflix"
,Q11_3 as "Meets needs- Amazon Prime "
,Q11_4 as "Meets needs- Now TV"
,Q11_5 as "Meets needs- Disney Plus"
,Q11_6 as "Meets needs- Apple TV"
,Q11a_1 as "Brand for me- BritBox"
,Q11a_2 as "Brand for me- Netflix"
,Q11a_3 as "Brand for me- Amazon Prime "
,Q11a_4 as "Brand for me- Now TV"
,Q11a_5 as "Brand for me- Disney Plus"
,Q11a_6 as "Brand for me- Apple TV"
,Q12_1_1 as "Brand statements- Has something I want to watch - Netflix"
,Q12_1_2 as "Brand statements- Has something I want to watch - BritBox"
,Q12_1_3 as "Brand statements- Has something I want to watch - Amazon Prime"
,Q12_1_4 as "Brand statements- Has something I want to watch - Now TV"
,Q12_1_5 as "Brand statements- Has something I want to watch - None"
,Q12_1_6 as "Brand statements- Has something I want to watch - Apple TV"
,Q12_2_1 as "Brand statements- Has quality programmes  - Netflix"
,Q12_2_2 as "Brand statements- Has quality programmes - BritBox"
,Q12_2_3 as "Brand statements- Has quality programmes - Amazon Prime"
,Q12_2_4 as "Brand statements- Has quality programmes - Now TV"
,Q12_2_5 as "Brand statements- Has quality programmes - None"
,Q12_2_6 as "Brand statements- Has quality programmes - Apple TV"
,Q12_3_1 as "Brand statements- Has programmes you wont find elsewhere  - Netflix"
,Q12_3_2 as "Brand statements- Has programmes you wont find elsewhere - BritBox"
,Q12_3_3 as "Brand statements- Has programmes you wont find elsewhere - Amazon Prime"
,Q12_3_4 as "Brand statements- Has programmes you wont find elsewhere - Now TV"
,Q12_3_5 as "Brand statements- Has programmes you wont find elsewhere - None"
,Q12_3_6 as "Brand statements- Has programmes you wont find elsewhere - Apple TV"
,Q12_4_1 as "Brand statements- Has a good range of programmes available  - Netflix"
,Q12_4_2 as "Brand statements- Has a good range of programmes available - BritBox"
,Q12_4_3 as "Brand statements- Has a good range of programmes available - Amazon Prime"
,Q12_4_4 as "Brand statements- Has a good range of programmes available - Now TV"
,Q12_4_5 as "Brand statements- Has a good range of programmes available - None"
,Q12_4_6 as "Brand statements- Has a good range of programmes available - Apple TV"
,Q12_5_1 as "Brand statements- Has a large catalogue of programmes  - Netflix"
,Q12_5_2 as "Brand statements- Has a large catalogue of programmes - BritBox"
,Q12_5_3 as "Brand statements- Has a large catalogue of programmes - Amazon Prime"
,Q12_5_4 as "Brand statements- Has a large catalogue of programmes - Now TV"
,Q12_5_5 as "Brand statements- Has a large catalogue of programmes - None"
,Q12_5_6 as "Brand statements- Has a large catalogue of programmes - Apple TV"
,Q12_6_1 as "Brand statements- Makes its own programmes  - Netflix"
,Q12_6_2 as "Brand statements- Makes its own programmes - BritBox"
,Q12_6_3 as "Brand statements- Makes its own programmes - Amazon Prime"
,Q12_6_4 as "Brand statements- Makes its own programmes - Now TV"
,Q12_6_5 as "Brand statements- Makes its own programmes - None"
,Q12_6_6 as "Brand statements- Makes its own programmes - Apple TV"
,Q12_7_1 as "Brand statements- Programmes updated every week  - Netflix"
,Q12_7_2 as "Brand statements- Programmes updated every week - BritBox"
,Q12_7_3 as "Brand statements- Programmes updated every week - Amazon Prime"
,Q12_7_4 as "Brand statements- Programmes updated every week - Now TV"
,Q12_7_5 as "Brand statements- Programmes updated every week - None"
,Q12_7_6 as "Brand statements- Programmes updated every week - Apple TV"
,Q12_8_1 as "Brand statements- Is ad free  - Netflix"
,Q12_8_2 as "Brand statements- Is ad free - BritBox"
,Q12_8_3 as "Brand statements- Is ad free - Amazon Prime"
,Q12_8_4 as "Brand statements- Is ad free - Now TV"
,Q12_8_5 as "Brand statements- Is ad free - Apple TV"
,Q12_8_6 as "Brand statements- Is ad free - None"
,Q12_9_1 as "Brand statements- Is good value for money  - Netflix"
,Q12_9_2 as "Brand statements- Is good value for money - BritBox"
,Q12_9_3 as "Brand statements- Is good value for money - Amazon Prime"
,Q12_9_4 as "Brand statements- Is good value for money - Now TV"
,Q12_9_5 as "Brand statements- Is good value for money - Apple TV"
,Q12_9_6 as "Brand statements- Is good value for money - None"
,Q13a_1 as "Britishness Statements without BVOD- It has the best of British talent"
,Q13a_2 as "Britishness Statements without BVOD- It is the best source for British TV"
,Q13a_3 as "Britishness Statements without BVOD- It is the best source for British Boxsets"
,Q13a_4 as "Britishness Statements without BVOD- It has the best American talent"
,Q13a_5 as "Britishness Statements without BVOD- It has the best International talent"
,Q13a_6 as "Britishness Statements without BVOD- It is the best source for American TV"
,Q13a_7 as "Britishness Statements without BVOD- It is the best source for International TV"
,Q13a_8 as "Britishness Statements without BVOD- It is the best source for American boxsets"
,Q13a_9 as "Britishness Statements without BVOD- It is the best source for International boxsets"
,Q13b_1 as "Britishness Statements with BVOD- It has the best of British talent"
,Q13b_2 as "Britishness Statements with BVOD- It is the best source for British TV"
,Q13b_3 as "Britishness Statements with BVOD- It is the best source for British Boxsets"
,Q13b_4 as "Britishness Statements with BVOD- It has the best American talent"
,Q13b_5 as "Britishness Statements with BVOD- It has the best International talent"
,Q13b_6 as "Britishness Statements with BVOD- It is the best source for American TV"
,Q13b_7 as "Britishness Statements with BVOD- It is the best source for International TV"
,Q13b_8 as "Britishness Statements with BVOD- It is the best source for American boxsets"
,Q13b_9 as "Britishness Statements with BVOD- It is the best source for International boxsets"
,Q14a_1 as "Device ownership- PC/ Laptop/ Mac"
,Q14a_2 as "Device ownership- Apple iPhone"
,Q14a_3 as "Device ownership- Apple tabletiPad/ iPad mini/ iPad pro"
,Q14a_4 as "Device ownership- Android smartphone"
,Q14a_5 as "Device ownership- Android tablet"
,Q14a_6 as "Device ownership- Amazon Fire tabletpreviously Kindle fire tablet"
,Q14a_7 as "Device ownership- PS3/ PS4"
,Q14a_8 as "Device ownership- Xbox 360/ Xbox One"
,Q14a_9 as "Device ownership- Smart TVwith an internet set-top-box built int..."
,Q14a_10 as "Device ownership- A Smart Speakere.g. Amazon echo, Google Home etc."
,Q14a_11 as "Device ownership- Other smartphone"
,Q14a_12 as "Device ownership- Other tablet"
,Q14a_13 as "Device ownership- Other games console"
,Q14a_14 as "Device ownership- None of these"
,Q14a_15 as "Device ownership- Don’t know"
,Q14b_1 as "TV device ownership- BT Vision"
,Q14b_2 as "TV device ownership- Freesat"
,Q14b_3 as "TV device ownership- Sky"
,Q14b_4 as "TV device ownership- TalkTalk TV"
,Q14b_5 as "TV device ownership- Virgin"
,Q14b_6 as "TV device ownership- YouView box"
,Q14b_7 as "TV device ownership- Apple TV box"
,Q14b_8 as "TV device ownership- Chromecast"
,Q14b_9 as "TV device ownership- Amazon Fire Stick"
,Q14b_10 as "TV device ownership- Roku box"
,Q14b_11 as "TV device ownership- NOW TV box or NOW TV Stick"
,Q14b_12 as "TV device ownership- Other Freeview box"
,Q14b_13 as "TV device ownership- None of these"
,Q14b_14 as "TV device ownership- Don’t know"
,Q15 as "Smart TV brand ownership"
,Q15_other as "Smart TV brand ownership"

from Brand_Tracker_{}_1
'''.format(This_month,This_month,This_month,This_month)


In [7]:
query=conn.cursor()
try:
    query.execute('''drop table Brand_Tracker_{}_2'''.format(This_month))
except:
    pass
query.execute(sql_query_string)
query.fetchall()
query.close()

    

In [8]:
mytab=pd.read_sql_query('''select * from Brand_Tracker_{}_2'''.format(This_month), conn)


### Create a function for repeated use throughout which allows a list of SQL lines of codes to be aggregated and executed

In [9]:
def create_table_from_variable_list(tabname,tabnum,script_list,fromnum=None):

    if fromnum==None:
        fromnum=tabnum-1
    
    reformatted_table_script='create table {}_{} as select '.format(tabname,tabnum)+\
                               ','.join(script_list)+\
    ' from {}_{}'.format(tabname,fromnum)


    query=conn.cursor()
    try:
        query.execute('''drop table {}_{}'''.format(tabname,tabnum))
    except:
        pass
    query.execute(reformatted_table_script)
    query.fetchall()
    query.close()
    print("Successfully created table: {}_{} from {}_{}".format(tabname,tabnum,tabname,fromnum))



In [1]:
# Concatenate create..select, statement list, and FROM statements
# Execute

my_list=[' select a','b as new_b','etc']
print(my_list)
print(','.join(my_list))


### Script 2: Replace all instances of selected/not selected with 1/0

 Loop through all variables in the table, returning unique values 
 If variable contains selected/not selected then generate case statement, otherwise take original var name into a list


In [11]:
re_formatted_table_script=[]

for x in mytab.columns:
    unique_vals=pd.read_sql_query('''select distinct "{}" from Brand_Tracker_{}_2 order by 1'''.format(x,This_month), conn)[x].to_list()
    
    if 'selected' in unique_vals and None in unique_vals and 'not selected' in unique_vals:
        script='''case when "{}" =\'selected\' then 1 
                        when "{}" =\'not selected\' then 0
                        end as "{}"'''.format(x,x,x)
    elif 'selected' in unique_vals and None in unique_vals:
        script='''case when "{}" =\'selected\' then 1 else 0 end as "{}"'''.format(x,x)
    elif 'selected' in unique_vals and 'not selected' in unique_vals:
         script='''case when "{}" =\'selected\' then 1 
                        when "{}" =\'not selected\' then 0
                        end as "{}"'''.format(x,x,x)
    else:
        script='''"{}"'''.format(x)
    
    re_formatted_table_script.append(script)

create_table_from_variable_list('Brand_tracker_{}'.format(This_month),3,re_formatted_table_script)

### Script 3: Standardise variable names

In [12]:
varlist=[]
mytab=pd.read_sql_query('''select * from Brand_Tracker_{}_3'''.format(This_month), conn)
for x in mytab.columns:
    # Standardise the naming of Amazon
    x1=x.replace('Amazon Prime Video','Amazon Prime')
    x2=x1.replace('All 4','All4')
    x3=x2.replace('Hub+','Hub Plus')
    x4=x3.replace('Apple TV+','Apple TV Plus')
    x5=x4.replace('Apple TV +','Apple TV Plus')
    x6=x5.replace('ITV player/hub/OD','ITV Hub')
    x7=x6.replace('Apple TV/Apple Masterbrand','Apple TV')
    
    varlist.append('''"{}" as "{}"'''.format(x,x7))
create_table_from_variable_list('Brand_tracker_{}'.format(This_month),4,varlist)
    

### Script 4: Create awareness fields

In [13]:
varlist=['*']
mytab=pd.read_sql_query('''select * from Brand_Tracker_{}_4'''.format(This_month), conn)
for x in mytab.columns:
    
    if x.split('-')[0]=='Experience with brand':
        varlist.append('''case when "{}" in 
        ('Have used it but not within the last 3 months', 
        'Have used it within the last 3 months, but not within the last month', 
        'Have used it within the last month', 
        'Heard of it but never used') then 1
        when "{}" = 'Never heard of it' then 0 end as "Awareness-{}"'''.format(x,x,x.split('-')[1]))
        
        varlist.append('''case when "{}" in 
        ('Have used it but not within the last 3 months', 
        'Have used it within the last 3 months, but not within the last month', 
        'Have used it within the last month'
        ) then 1
        when "{}"  in ('Heard of it but never used','Never heard of it') then 0 end as "Ever Used-{}"'''.format(x,x,x.split('-')[1]))
        
        varlist.append('''case when "{}" in ('Have used it within the last month') then 1
        when "{}"  in ('Have used it but not within the last 3 months', 
        'Have used it within the last 3 months, but not within the last month',
        'Heard of it but never used','Never heard of it') then 0 end as "Used Past Month-{}"'''.format(x,x,x.split('-')[1]))
       
    if x.split('-')[0]=='Likeability':
        varlist.append('''case 
        when "{}" in (0,1,2) then '0-2'
        when "{}" in (3,4,5) then '3-5'
        when "{}" in (6,7,8) then '6-8'
        when "{}" in (9,10) then '9-10'
        end as "Likeability Band-{}"'''.format(x,x,x,x,x.split('-')[1]))

    if x.split('-')[0].strip()=='Uniqueness':
        varlist.append('''case 
        when "{}" in (0,1,2) then '0-2'
        when "{}" in (3,4,5) then '3-5'
        when "{}" in (6,7,8) then '6-8'
        when "{}" in (9,10) then '9-10'
        end as "Uniqueness Band-{}"'''.format(x,x,x,x,x.split('-')[1]))
              
    if x.split('-')[0].strip()=='Recommendation':
        varlist.append('''case 
        when "{}" in (0,1,2) then '0-2'
        when "{}" in (3,4,5) then '3-5'
        when "{}" in (6,7,8) then '6-8'
        when "{}" in (9,10) then '9-10'
        end as "Recommendation Band-{}"'''.format(x,x,x,x,x.split('-')[1]))    
        
        varlist.append('''case 
        when "{}" in (0,1,2,3,4,5,6) then 'Detractor'
        when "{}" in (7,8) then 'Neutral'
        when "{}" in (9,10) then 'Promoter'
        end as "NPS Band-{}"'''.format(x,x,x,x.split('-')[1]))  
        
    if x.split('-')[0].strip()=='Brand for me':
        varlist.append('''case 
        when "{}" in (0,1,2) then '0-2'
        when "{}" in (3,4,5) then '3-5'
        when "{}" in (6,7,8) then '6-8'
        when "{}" in (9,10) then '9-10'
        end as "Brand for me Band-{}"'''.format(x,x,x,x,x.split('-')[1]))
        
    if x.split('-')[0].strip()=='ITP paid v2 scale':
        varlist.append('''case 
        when "{}" in (0,1,2,3) then '0-3'
        when "{}" in (4,5,6,7) then '4-7'
        when "{}" in (8,9,10) then '8-10'
        end as "ITP Band-{}"'''.format(x,x,x,x.split('-')[1]))
        
create_table_from_variable_list('Brand_tracker_{}'.format(This_month),5,varlist)     

In [14]:
mytab=pd.read_sql_query('''select * from Brand_Tracker_{}_5'''.format(This_month), conn)


In [15]:
for n,x in enumerate(mytab.columns):
    if x=='weight':
        pass
    else:
        if mytab.dtypes[n]=='float64':
            print('"{}" is a float'.format(x))
        elif mytab.dtypes[n]=='int64':
            print('"{}" is an int'.format(x))
        else:
            print('"{}" is neither'.format(x))
          

### Script 5: Weight all numeric variables

In [16]:
varlist=[]
for n,x in enumerate(mytab.columns):
    if x=='weight':
        varlist.append('"{}"'.format(x))
    else:
        if mytab.dtypes[n] in ('float64','int64'):
            varlist.append('cast("{}" as real)*weight as "{}"'.format(x,x))            
            varlist.append('"{}" as "Unweighted- {}"'.format(x,x))
        else:
            varlist.append('"{}"'.format(x))

create_table_from_variable_list('Brand_tracker_{}'.format(This_month),6,varlist)     

In [17]:
for x in varlist:
    print(x)

### Script 6: Transform table to put brand battery on lines rather than columns

In [18]:
mytab=pd.read_sql_query('''select * from Brand_Tracker_{}_6'''.format(This_month), conn)

################################
# This loops through the questions which have the brands listed above in their text as an "option" in the dash-delimited
# format. If they are there, they go into a list of questions we want to standardise, if not they go into a set to leave alone
################################

exclude_qs=['TV device ownership- Apple TV box'] # variables which might accidentally be picked up by the brand battery but we don't want to be
pivotable_qs=[]
retain_qs=[]
for x in mytab.columns:
    match=None
    for opt in brand_battery:
        if opt in [i.strip() for i in x.split("-")]:
            match=x
    if match:
        if x in exclude_qs:
            retain_qs.append(x)
        else:
            pivotable_qs.append(x)
    else:
        retain_qs.append(x)


In [19]:
##### For the next step we will make use of the SQL "union" function when duplicating info for each brand. 
##### However not every question is asked of every brand 
##### For a UNION statement to work, there needs to be an equal number of columns, so for the questions not asked
##### (probably because they're not applicable) we nonetheless have to create the q
##### Therefore we create a 'stemlist' which holds all the questions as they are without the brands, for example:
##### "Awareness- Netflix" and "Awareness- BritBox" have a single 'stem' of "Awareness"
##### This means if e.g. awareness was not asked of a brand "Y" we can create "Awareness- Y" as a column and leave it blank
##### To make the code work

stemlist=[]
for x in brand_battery:
    # Loop through the pivotable questions
    for y in pivotable_qs:
        q_list=[i.strip() for i in y.split("-")]
        if x.strip() in q_list:
            q_list.remove(x)
            var_name='-'.join(q_list)
            stemlist.append(var_name)
stemlist=list(set(stemlist))


In [20]:
script_to_run='''create table Brand_tracker_{}_7 as '''.format(This_month)

for x in brand_battery: # For every brand
    varlist=[]
    varlist.append('''select '{}' as Brand'''.format(x)) # Add the create table statement
    for y in stemlist: # For every question type asked of multiple brands
        found=0 
        for z in pivotable_qs:
            q_list=[i.strip() for i in z.split("-")] # Break question into component parts
            if x in q_list: #If brand is in the question
                q_list.remove(x) # Remove it from the list
                var_name='-'.join(q_list) # Create the "stem"
                if var_name==y: #If the stems match:
                    varlist.append('"{}" as "{}"'.format(z,var_name))
                    found=1
        if found==0:
            varlist.append('Null as "{}"'.format(y))
    
    for r in retain_qs:
        varlist.append('"{}"'.format(r))

    script_to_run=script_to_run+','.join(varlist)+' from Brand_tracker_{}_6 union all '.format(This_month)
            
script_to_run=script_to_run[:-10]

In [21]:
query=conn.cursor()
try:
    query.execute('''drop table Brand_tracker_{}_7'''.format(This_month))
except:
    pass
query.execute(script_to_run)
query.fetchall()
query.close()

mytab=pd.read_sql_query('''select * from Brand_Tracker_{}_7'''.format(This_month), conn)


In [22]:
tg.Export_dataframe_to_excel\
(mytab,r"/Users/stepwate/Desktop/Brand Tracker/Brand Battery {}".format(This_month),100,\
 export_index=False,Include_date='N')

